<a href="https://colab.research.google.com/github/YumingxuanGuo/Data-Handler/blob/main/courses_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd

year_term = "2022/spring"
school_url = "https://courses.illinois.edu/schedule/" + year_term

response = requests.get(school_url)

department_name = []

page = soup(response.text, 'html.parser')

for link in page.find_all('a'):
  candidates = str(link.get('href'))
  if (candidates.find('/schedule/2022/spring/') != -1):
    department_name.append(candidates[22:])

In [23]:
import multiprocessing
from multiprocessing import Pool
import sys

MAX_WORKER_NUM = multiprocessing.cpu_count()

def job(diction, url, dep):
  response_coures_list = requests.get(url)
  page_courses_list = soup(response_coures_list.text, 'html.parser')
  courses_name_nums = dict()
  for st_a in page_courses_list.find_all('a'):
    nums_pre = str(st_a.get('href'))
    if ((year_term + '/' + dep) in nums_pre and 'pdf' not in nums_pre):
      num = str(st_a.get('href'))[-3:]
      name = st_a.string
      courses_name_nums[num] = name
  diction[dep] = courses_name_nums

url_list = []
manager = multiprocessing.Manager()
all_data = manager.dict()

for dep in department_name:
  dep_url = school_url + '/' + dep
  url_list.append(dep_url)

p = Pool(15)
sys.setrecursionlimit(2500000)
for i in range(len(url_list)):
  p.apply_async(job, args = (all_data, url_list[i], department_name[i]))

p.close()
p.join()
print(all_data.keys())

['AAS', 'AGCM', 'AFST', 'AFAS', 'AGED', 'ABE', 'ADV', 'AFRO', 'AIS', 'AE', 'ALEC', 'ACCY', 'AHS', 'ACE', 'ACES', 'ANSC', 'ANTH', 'ARAB', 'ARCH', 'ART', 'ARTD', 'ARTE', 'ARTJ', 'ASST', 'ARTS', 'ARTF', 'ARTH', 'ATMS', 'ASRM', 'ASTR', 'BCOG', 'BCS', 'BADM', 'BDI', 'BIOC', 'BIOE', 'CDB', 'CHBE', 'BIOP', 'BUS', 'BIOL', 'CB', 'BTW', 'BSE', 'CEE', 'CHIN', 'CHEM', 'CHP', 'CHLH', 'CI', 'CIC', 'CLCV', 'CSE', 'CMN', 'CW', 'CPSC', 'DTX', 'CZCH', 'CWL', 'CS', 'CLE', 'DANC', 'EALC', 'EDPR', 'ECON', 'ECE', 'EDUC', 'EIL', 'ENG', 'ENSU', 'ERAM', 'ESE', 'ENT', 'ENVS', 'EPOL', 'EPSY', 'FAA', 'ESL', 'EURO', 'ENGL', 'FLTE', 'FR', 'FIN', 'FSHN', 'GC', 'GSD', 'GS', 'GMC', 'GRK', 'GRKM', 'GLBL', 'HEBR', 'GEOG', 'GER', 'GWS', 'GEOL', 'HDFS', 'HIST', 'HORT', 'HNDI', 'HT', 'HUM', 'IB', 'IE', 'JAPN', 'JS', 'IHLT', 'KOR', 'INFO', 'ITAL', 'KIN', 'JOUR', 'IS', 'LAS', 'LAST', 'LAT', 'LA', 'LCTL', 'LEAD', 'MFST', 'MDVL', 'LER', 'MDIA', 'MBA', 'MICR', 'LING', 'MCB', 'MACS', 'LLS', 'LAW', 'MATH', 'ME', 'MILS', 'MIP', 'M

In [24]:
df = pd.DataFrame()
for i in all_data.items():
  temp_list = []
  for j in i[1].keys():
    temp_list.append(j + " " + i[1][j])
  temp = pd.DataFrame({i[0]: temp_list})
  df = pd.concat([df, temp], axis = 1)

df.to_csv("courses_num_name.csv")

In [ ]:
import re
import json
import os

if not(os.path.exists('course_info')):
  os.mkdir('course_info')

def job_2(department):
  department_dataframe = pd.dataFrame()
  for course_number in all_data[department]:
    course_url = school_url + '/' + department + '/' + course_number
    response = requests.get(course_url)
    text = response.text
    page = soup(text, 'html.parser')
    pattern = re.compile(r"var sectionDataObj = (.*?);$", re.MULTILINE | re.DOTALL)
    script = page.find("script", text = pattern)
    info = pattern.search(script.prettify()).group(1)
    data = json.loads(info)
    crn = []
    type_ = []
    section = []
    time = []
    day = []
    location = []
    instructor = []
    for course in data:
      crn.append(course["crn"].strip())
      type_.append(course["type"].split("meeting\">")[1].split("</div>")[0].strip())
      section.append(course["section"].split("meeting\">")[1].split("</div>")[0].strip())
      time.append(course["time"].split("meeting\">")[1].split("</div>")[0].strip())
      day.append(course["day"].split("meeting\">")[1].split("</div>")[0].strip())
      location.append(course["location"].split("meeting\">")[1].split("</div>")[0].strip())
      instr_info = course["instructor"].split("meeting\">")[1].split("</div>")[0].strip()
      if (instr_info):
        instr_info = instr_info[:-6].replace("<br />", " & ", 10)
      instructor.append(instr_info)
    course_data = pd.DataFrame({"Course":department+course_number, "Name":all_data[department][course_number].strip(), "CRN":crn, "Type":type_, "Section":section, "Time":time, "Day":day, "Location":location, "Instructor":instructor})
    department_dataframe = pd.concat([department_dataframe, course_data], ignore_index = True)
  department_dataframe.to_csv("course_info/" + department + ".csv")

p = Pool(16)
sys.setrecursionlimit(2500000)
for i in range(len(department_name)):
  p.apply_async(job_2, (department_name[i],))
p.close()
p.join()

!zip -r /content/course_info.zip /content/course_info